# Import packages

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break


%load_ext autoreload
%autoreload 2

import sys
from data_wrangling import specific_utils, process_monkey_information, base_processing_class
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils, plot_cluster_replacement
from decision_making_analysis.decision_making import decision_making_utils, plot_decision_making, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_helper_class, GUAT_collect_info_class, GUAT_combine_info_class, add_features_GUAT_and_TAFT
from decision_making_analysis import free_selection, replacement, trajectory_info
from visualization.matplotlib_tools import plot_trials, plot_polar, additional_plots, plot_behaviors_utils, plot_statistics
from visualization.animation import animation_func, animation_utils, animation_class
from null_behaviors import sample_null_distributions, show_null_trajectory
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from machine_learning.RL.env_related import env_for_lstm, env_utils, base_env, collect_agent_data_utils
from machine_learning.RL.lstm import GRU_functions, LSTM_functions, LSTM_functions, lstm_for_multiff_class
from machine_learning.RL.SB3 import interpret_neural_network, sb3_for_multiff_class, rl_for_multiff_utils, SB3_functions
from eye_position_analysis import eye_positions
from neural_data_analysis.neural_analysis_tools.model_neural_data import neural_data_modeling
from decision_making_analysis.compare_GUAT_and_TAFT import find_GUAT_or_TAFT_trials
from machine_learning.RL.SB3 import rl_for_multiff_utils, rl_for_multiff_class
from machine_learning.RL.ff_attention import attn_sac_ff, attn_sac_rnn, env_attn_multiff

import os, sys
import torch
import numpy as np
import pickle
from gymnasium import spaces, Env
import torch
import optuna
from numpy import pi
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from torch.linalg import vector_norm
from IPython.display import HTML
from functools import partial
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
import gc
from importlib import reload
plt.rcParams["animation.html"] = "html5"
torch.set_printoptions(sci_mode=False)
np.set_printoptions(suppress=True)

device_idx = 0
# device = torch.device("cuda:" + str(device_idx) if torch.cuda.is_available() else "cpu")
## if using Jupyter Notebook
device = "mps" if torch.backends.mps.is_available() else "cpu"
model_folder_name = "RL_models/LSTM_stored_models/all_agents/gen_0/LSTM_Aug_1_24"
os.makedirs(model_folder_name, exist_ok=True)
PLAYER = "agent"

In [ ]:
stop!

# streamline everything

In [ ]:
# env for curriculum training...

env_kwargs = {
    "reward_per_ff": 80,

    "dv_cost_factor": 0,
    "dw_cost_factor": 0,
    "w_cost_factor": 0,
    "dt": 0.25,
    "flash_on_interval": 2.1
}

### env1

In [ ]:
lm.env_kwargs

In [ ]:
reload(env_for_lstm)
reload(sb3_for_multiff_class)
reload(rl_for_multiff_class)

In [ ]:
params_file = os.path.join('RL_models/LSTM_stored_models/all_agents/gen_9_determ_eval/dv0_dw0_w0_mem0', 'env_params.txt')

import pandas as pd
params_file = pd.read_csv(params_file)
params_file
# lm = lstm_for_multiff_class.LSTMforMultifirefly()

In [ ]:
# lm = lstm_for_multiff_class.LSTMforMultifirefly()

env_kwargs = {'num_obs_ff': 2,
              'add_action_to_obs': False,
              'angular_terminal_vel': 1,
              "dt": 0.25,
              "flash_on_interval": 0.3,
              "dv_cost_factor": 0,
              "dw_cost_factor": 0,
              "w_cost_factor": 0,
            }   
lm = lstm_for_multiff_class.LSTMforMultifirefly(overall_folder='RL_models/LSTM_stored_models/all_agents/gen_9_determ_eval/',
                                                **env_kwargs)

                                             

lm.streamline_everything(currentTrial_for_animation=None, num_trials_for_animation=None, duration=[10, 40],
                         best_model_after_curriculum_exists_ok=True)

In [ ]:
lm.env.num_elem_per_ff

In [ ]:
lm.env.obs_space_length

In [ ]:
lm.env_kwargs

# Feed-forward SAC

In [ ]:
!pwd

In [ ]:
/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/multiff_code/test_env_debug.py
from test_env_debug import test_env_debug

In [ ]:
from multiff_analysis.multiff_code import test_env_debug

In [ ]:
test_env_debug.main()

In [ ]:
reload(base_env)

In [ ]:

# 1) Make env wrapper (inherits your MultiFF)
env = env_attn_multiff.EnvForAttentionSAC(
    num_obs_ff=5,
    slot_fields=['valid','d_log','sin','cos','t_start_seen'],  # choose fields by name
    recenter=True,                                             # map to [-1,1]
    flash_on_interval=0.3, episode_len=1024
)

# 2) Build trainer
agent = attn_sac_ff.AttnSACforMultifirefly(model_folder='RL_models/ATTN_FF/agent_0/',
                               num_obs_ff=5, slot_fields=['valid','d_log','sin','cos','t_start_seen'])

agent.make_env()  # pass/override env kwargs here if you want
agent.make_agent(
    k_top=4, d_slot=64, n_heads=2, include_ctx=True,
    attn_temperature=1.0, slot_dropout_p=0.05, gumbel_topk=True,
    lr=3e-4, gamma=0.99, tau=0.995, target_entropy=-2.0,
    batch_size=128, replay_size=200_000, random_steps=2_000, updates_per_step=1,
    amp=False, log_every=1000, eval_every=0, save_every=0
)

# 3) Train
agent.regular_training(total_env_steps=50_000)

# 4) Save / Eval
agent.save_agent()
agent.test_agent(num_eps=3, deterministic=True)


# RNN

In [ ]:
rnn_agent = attn_sac_rnn.AttnRNNSACforMultifirefly(model_folder='RL_models/ATTN_RNN/agent_0/',
                                      num_obs_ff=5, slot_fields=['valid','d_log','sin','cos','t_start_seen'])
rnn_agent.make_env()
rnn_agent.make_agent(
    k_top=4, d_slot=64, include_ctx=True, rnn='gru', d_hidden=256,
    attn_temperature=1.0, slot_dropout_p=0.05, gumbel_topk=True,
    lr=3e-4, gamma=0.99, tau=0.995, target_entropy=-2.0,
    batch_size=16, seq_len_total=128, burn_in=16, updates_per_step=1,
    amp=False, log_every=1000, eval_every=0
)
rnn_agent.regular_training(total_env_steps=50_000)
rnn_agent.save_agent()
rnn_agent.test_agent(num_eps=3, deterministic=True)
